In [0]:
# pip install sodapy
%pip install sodapy


Python interpreter will be restarted.
Collecting sodapy
 Downloading sodapy-2.1.1-py2.py3-none-any.whl (14 kB)
Requirement already satisfied: requests>=2.20.0 in /databricks/python3/lib/python3.8/site-packages (from sodapy) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.20.0->sodapy) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.20.0->sodapy) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.20.0->sodapy) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.20.0->sodapy) (2.10)
Installing collected packages: sodapy
Successfully installed sodapy-2.1.1
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4a8d5af0-c058-49a9-bacb-15f33913f17a/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
#import libraries
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql import Row
from pyspark import SparkFiles
import pandas as pd
from sodapy import Socrata

In [0]:
# create query for Socrata
query = """
select 

accident_date, age_at_injury, average_weekly_wage, birth_year, carrier_name, claim_identifier, claim_injury_type, claim_type, closed_count, current_claim_status, gender, injured_in_county_name, wcio_cause_of_injury_code, wcio_cause_of_injury_desc, wcio_nature_of_injury_code, wcio_nature_of_injury_desc, wcio_pob_code, wcio_pob_desc, industry_code, industry_desc

where
    accident_date between '2022-01-01T00:00:00' and '2022-12-31T11:59:59'
    
"""

In [0]:
# calling Socrata API with query
client = Socrata("data.ny.gov", '<Token>')


results = client.get("jshw-gkgu",query = query)

# Convert to pandas DataFrame
spark_df = spark.createDataFrame(results)

In [0]:
spark_df.printSchema()

root
-- accident_date: string (nullable = true)
-- age_at_injury: string (nullable = true)
-- average_weekly_wage: string (nullable = true)
-- birth_year: string (nullable = true)
-- carrier_name: string (nullable = true)
-- claim_identifier: string (nullable = true)
-- claim_injury_type: string (nullable = true)
-- claim_type: string (nullable = true)
-- closed_count: string (nullable = true)
-- current_claim_status: string (nullable = true)
-- gender: string (nullable = true)
-- industry_code: string (nullable = true)
-- industry_desc: string (nullable = true)
-- injured_in_county_name: string (nullable = true)
-- wcio_cause_of_injury_code: string (nullable = true)
-- wcio_cause_of_injury_desc: string (nullable = true)
-- wcio_nature_of_injury_code: string (nullable = true)
-- wcio_nature_of_injury_desc: string (nullable = true)
-- wcio_pob_code: string (nullable = true)
-- wcio_pob_desc: string (nullable = true)

In [0]:
# save to delta table as an update
spark_df.write.format("delta").mode("overwrite").saveAsTable("default.NY_workers_compensation_update")

In [0]:
%sql
select *
from NY_workers_compensation_update
limit 5

accident_date,age_at_injury,average_weekly_wage,birth_year,carrier_name,claim_identifier,claim_injury_type,claim_type,closed_count,current_claim_status,gender,industry_code,industry_desc,injured_in_county_name,wcio_cause_of_injury_code,wcio_cause_of_injury_desc,wcio_nature_of_injury_code,wcio_nature_of_injury_desc,wcio_pob_code,wcio_pob_desc
2022-01-01T00:00:00.000,74,0.00,1947,STATE INSURANCE FUND,5898543,2. NON-COMP,WORKERS COMPENSATION CLAIM,0,NEWLY ASSEMBLED,F,53,REAL ESTATE AND RENTAL AND LEASING,BRONX,31,"FALL, SLIP OR TRIP, NOC",52,STRAIN OR TEAR,53,KNEE
2022-01-01T00:00:00.000,58,0.00,0,STATE INSURANCE FUND,5956737,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,HEARING UNSET,M,33,MANUFACTURING,QUEENS,97,REPETITIVE MOTION,52,STRAIN OR TEAR,35,HAND
2022-01-01T00:00:00.000,64,0.00,1957,LM INSURANCE CORP,5916708,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,NO FURTHER ACTION,F,62,HEALTH CARE AND SOCIAL ASSISTANCE,KINGS,88,NATURAL DISASTERS,83,COVID-19,91,BODY SYSTEMS AND MULTIPLE BODY SYSTEMS
2022-01-01T00:00:00.000,20,0.00,2001,TECHNOLOGY INSURANCE CO INC,5895055,2. NON-COMP,WORKERS COMPENSATION CLAIM,0,NEWLY ASSEMBLED,F,62,HEALTH CARE AND SOCIAL ASSISTANCE,SUFFOLK,56,LIFTING,52,STRAIN OR TEAR,42,LOWER BACK AREA
2022-01-01T00:00:00.000,40,0.00,1981,STATE INSURANCE FUND,5906295,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,NO FURTHER ACTION,M,92,PUBLIC ADMINISTRATION,DUTCHESS,33,ON STAIRS,52,STRAIN OR TEAR,53,KNEE


In [0]:
# merge and update/insert new records from update table
%sql
MERGE INTO NY_workers_compensation_delta
USING NY_workers_compensation_update
ON NY_workers_compensation_delta.claim_identifier = NY_workers_compensation_update.claim_identifier
WHEN MATCHED THEN
  UPDATE SET
    claim_identifier = NY_workers_compensation_update.claim_identifier,
    accident_date = NY_workers_compensation_update.accident_date,
    age_at_injury = NY_workers_compensation_update.age_at_injury,
    average_weekly_wage = NY_workers_compensation_update.average_weekly_wage,
    birth_year = NY_workers_compensation_update.birth_year,
    carrier_name = NY_workers_compensation_update.carrier_name,
    claim_injury_type = NY_workers_compensation_update.claim_injury_type,
    claim_type = NY_workers_compensation_update.claim_type,
    closed_count = NY_workers_compensation_update.closed_count,
    current_claim_status = NY_workers_compensation_update.current_claim_status,
    gender = NY_workers_compensation_update.gender,
    county_of_injury = NY_workers_compensation_update.injured_in_county_name,
    wcio_cause_of_injury_code = NY_workers_compensation_update.wcio_cause_of_injury_code,
    wcio_cause_of_injury_description = NY_workers_compensation_update.wcio_cause_of_injury_desc,
    wcio_nature_of_injury_code = NY_workers_compensation_update.wcio_nature_of_injury_code,
    wcio_nature_of_injury_description = NY_workers_compensation_update.wcio_nature_of_injury_desc,
    wcio_part_of_body_code = NY_workers_compensation_update.wcio_pob_code,
    wcio_part_of_body_description = NY_workers_compensation_update.wcio_pob_desc,
    industry_code = NY_workers_compensation_update.industry_code,
    industry_code_description = NY_workers_compensation_update.industry_desc
    
WHEN NOT MATCHED
  THEN INSERT (
    claim_identifier,
    accident_date, 
    age_at_injury, 
    average_weekly_wage, 
    birth_year, 
    carrier_name, 
    claim_injury_type, 
    claim_type, 
    closed_count, 
    current_claim_status, 
    gender, 
    county_of_injury, 
    wcio_cause_of_injury_code, 
    wcio_cause_of_injury_description, 
    wcio_nature_of_injury_code, 
    wcio_nature_of_injury_description, 
    wcio_part_of_body_code, 
    wcio_part_of_body_description, 
    industry_code,
    industry_code_description
  )
  VALUES (
    NY_workers_compensation_update.claim_identifier,
    NY_workers_compensation_update.accident_date,
    NY_workers_compensation_update.age_at_injury,
    NY_workers_compensation_update.average_weekly_wage,
    NY_workers_compensation_update.birth_year,
    NY_workers_compensation_update.carrier_name,
    NY_workers_compensation_update.claim_injury_type,
    NY_workers_compensation_update.claim_type,
    NY_workers_compensation_update.closed_count,
    NY_workers_compensation_update.current_claim_status,
    NY_workers_compensation_update.gender,
    NY_workers_compensation_update.injured_in_county_name,
    NY_workers_compensation_update.wcio_cause_of_injury_code,
    NY_workers_compensation_update.wcio_cause_of_injury_desc,
    NY_workers_compensation_update.wcio_nature_of_injury_code,
    NY_workers_compensation_update.wcio_nature_of_injury_desc,
    NY_workers_compensation_update.wcio_pob_code,
    NY_workers_compensation_update.wcio_pob_desc,
    NY_workers_compensation_update.industry_code,
    NY_workers_compensation_update.industry_desc
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1000,1000,0,0


In [0]:
%sql

select *
from NY_workers_compensation_delta
limit 5

accident,accident_date,age_at_injury,alternative_dispute_resolution,ancr_date,assembly_date,attorney/representative,average_weekly_wage,birth_year,c-2_date,c-3_date,carrier_name,carrier_type,claim_identifier,claim_injury_type,claim_type,closed_count,controverted_date,county_of_injury,covid-19_indicator,current_claim_status,district_name,first_appeal_date,first_hearing_date,gender,hearing_count,highest_process,ime-4_count,industry_code,industry_code_description,interval_assembled_to_ancr,medical_fee_region,occupational_disease,oiics_event_exposure_code,oiics_event_exposure_description,oiics_injury_source_code,oiics_injury_source_description,oiics_nature_of_injury_code,oiics_nature_of_injury_description,oiics_part_of_body_code,oiics_part_of_body_description,oiics_secondary_source_code,oiics_secondary_source_description,ppd_non-scheduled_loss_date,ppd_scheduled_loss_date,ptd_date,section_32_date,wcio_cause_of_injury_code,wcio_cause_of_injury_description,wcio_nature_of_injury_code,wcio_nature_of_injury_description,wcio_part_of_body_code,wcio_part_of_body_description,zip_code
Y,07/09/2016,30,N,null,07/15/2016,N,0.0,1985,07/14/2016,null,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,4338692,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,null,OTSEGO,N,NO FURTHER ACTION,BINGHAMTON,null,null,F,0,1. NO RESOLUTIONS,null,48,TRANSPORTATION AND WAREHOUSING,null,I,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,17,OBJECT BEING LIFTED OR HANDLED,59,"ALL OTHER SPECIFIC INJURIES, NOC",42,LOWER BACK AREA,12116
Y,07/12/2016,51,N,null,07/15/2016,N,0.0,1965,07/14/2016,null,"LOWE'S HOME CENTERS, INC.",4A. SELF PRIVATE,4338693,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,null,BROOME,N,NO FURTHER ACTION,BINGHAMTON,null,null,M,0,1. NO RESOLUTIONS,null,33,MANUFACTURING,null,II,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,56,LIFTING,52,STRAIN OR TEAR,42,LOWER BACK AREA,13901
Y,07/12/2016,41,N,null,07/15/2016,N,0.0,1974,07/15/2016,null,NY LUMBERMENS INS. TRUST FUND,4A. SELF PRIVATE,4338694,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,null,CORTLAND,N,ADMINISTRATIVELY CLOSED,BINGHAMTON,null,null,M,0,1. NO RESOLUTIONS,null,32,MANUFACTURING,null,I,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,53,TWISTING,49,SPRAIN OR TEAR,55,ANKLE,13077
Y,07/06/2016,55,N,null,07/15/2016,N,0.0,1961,07/14/2016,null,SAFETY NATIONAL CASUALTY CORP,1A. PRIVATE,4338695,2. NON-COMP,WORKERS COMPENSATION CLAIM,1,null,BROOME,N,NO FURTHER ACTION,BINGHAMTON,null,null,F,0,1. NO RESOLUTIONS,null,44,RETAIL TRADE,null,II,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,19,"CUT, PUNCTURE, SCRAPE, NOC",43,PUNCTURE,33,LOWER ARM,13901
Y,07/07/2016,51,N,01/12/2017,07/15/2016,Y,757.73,1964,07/15/2016,10/19/2016,STATE INSURANCE FUND,2A. SIF,4338696,5. PPD SCH LOSS,WORKERS COMPENSATION CLAIM,4,null,SULLIVAN,N,NO FURTHER ACTION,BINGHAMTON,null,null,F,0,1. NO RESOLUTIONS,1,72,ACCOMMODATION AND FOOD SERVICES,181,I,N,null,null,null,null,null,null,null,null,null,null,null,08/22/2017,null,null,53,TWISTING,52,STRAIN OR TEAR,53,KNEE,12719
